##  Applying knowledge to other fields: Transfer learning
Transfer learning is a very convenient technique consisting in using a previously trained model, reusing the weights adjusted for a benchmark (in the case of image classification problems it is Imagenet), 
In this section we will gather images from two different flower types, coming from the flower17 dataset. 
It is a 17 category flower dataset with 80 images for each class. The flowers chosen are some common flowers in the UK. The images have large scale, pose and light variations and there are also classes with large varations of images within the class and close similarity to other classes.
In this case we will gather the first 2 classes (Daffodil and Coltsfoot), and build a classifier on top of the pretrained VGG16 network.

Previously, we will will do image data aughmentation, because the images quantity could be not enough to abstract all the elements of any species.
Let's start by importing all the needed libraries, including applications, preprocession, the checkpoint model and associated object, to allow saving the intermediate steps, and the cv2 and Numpy ones, for image processing and numerical base operation.

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.models import load_model
from keras.applications.vgg16 import VGG16, decode_predictions,preprocess_input
import cv2
from io import BytesIO
import numpy as np
import urllib
from PIL import Image

Using TensorFlow backend.


In this section we will define all the variables affectin the input, data sources, and training parameters.

In [0]:
img_width, img_height = 224, 224
train_data_dir = "train"
validation_data_dir = "validation"
nb_train_samples = 300
nb_validation_samples = 100 
batch_size = 16
epochs = 20

Now we will invoke the VGG16 pretrained model, not including the top flattening layers.

In [3]:
model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

58892288/58889256 [==============================] - 2s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()


Now its time to compile the model, and create the  image data authmentation object for the training and testing dataset

In [4]:
!wget https://s3.amazonaws.com/italia18/transfer_learning_dataset.zip

--2018-04-26 10:23:43--  https://s3.amazonaws.com/italia18/transfer_learning_dataset.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.33.186
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.33.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7848999 (7.5M) [application/zip]
Saving to: ‘transfer_learning_dataset.zip’

transfer_learning_d 100%[===================>]   7.49M  5.89MB/s    in 1.3s    

2018-04-26 10:23:45 (5.89 MB/s) - ‘transfer_learning_dataset.zip’ saved [7848999/7848999]



In [5]:
!unzip -o transfer_learning_dataset.zip

Archive:  transfer_learning_dataset.zip
   creating: test/
  inflating: test/butt2.jpg          
  inflating: test/butter1.jpg        
  inflating: test/coldsfoot.jpg      
  inflating: test/coltsfoot-flower.jpg  
  inflating: test/daff1.jpg          
  inflating: test/gaff2.jpg          
  inflating: test/galan.jpg          
  inflating: test/galant2.jpg        
   creating: train/
   creating: train/1/
  inflating: train/1/image_0001.jpg  
  inflating: train/1/image_0002.jpg  
  inflating: train/1/image_0003.jpg  
  inflating: train/1/image_0004.jpg  
  inflating: train/1/image_0005.jpg  
  inflating: train/1/image_0006.jpg  
  inflating: train/1/image_0007.jpg  
  inflating: train/1/image_0008.jpg  
  inflating: train/1/image_0009.jpg  
  inflating: train/1/image_0010.jpg  
  inflating: train/1/image_0011.jpg  
  inflating: train/1/image_0012.jpg  
  inflating: train/1/image_0013.jpg  
  inflating: train/1/image_0014.jpg  
  inflating: train/1/image_0015.jpg  
  inflating: train/1/i

In [0]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)



Now we will properly generate the new augmented data

In [7]:
!unzip -o transfer_learning_dataset.zip

Archive:  transfer_learning_dataset.zip
  inflating: test/butt2.jpg          
  inflating: test/butter1.jpg        
  inflating: test/coldsfoot.jpg      
  inflating: test/coltsfoot-flower.jpg  
  inflating: test/daff1.jpg          
  inflating: test/gaff2.jpg          
  inflating: test/galan.jpg          
  inflating: test/galant2.jpg        
  inflating: train/1/image_0001.jpg  
  inflating: train/1/image_0002.jpg  
  inflating: train/1/image_0003.jpg  
  inflating: train/1/image_0004.jpg  
  inflating: train/1/image_0005.jpg  
  inflating: train/1/image_0006.jpg  
  inflating: train/1/image_0007.jpg  
  inflating: train/1/image_0008.jpg  
  inflating: train/1/image_0009.jpg  
  inflating: train/1/image_0010.jpg  
  inflating: train/1/image_0011.jpg  
  inflating: train/1/image_0012.jpg  
  inflating: train/1/image_0013.jpg  
  inflating: train/1/image_0014.jpg  
  inflating: train/1/image_0015.jpg  
  inflating: train/1/image_0016.jpg  
  inflating: train/1/image_0017.jpg  
  infla

In [8]:
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

# Save the model according to the conditions  
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


Found 120 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


It's time to fit the new final layers for the model

In [0]:
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
nb_epoch = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [early])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., callbacks=[<keras.ca..., steps_per_epoch=18, epochs=20, validation_steps=100)`
  import sys


Epoch 1/20
 2/18 [==>...........................] - ETA: 6:56 - loss: 0.7705 - acc: 0.4375

Then let's try with a daffoil image, testing the output of the classifier, which should output an array close to the [1.,0.] value, indicating that the probability for the first option is very high.

In [0]:
#@title
im = cv2.resize(cv2.imread('test/gaff2.jpg'), (img_width, img_height))
im = np.expand_dims(im, axis=0).astype(np.float32)
im=preprocess_input(im)
print (im.shape)
out = model_final.predict(im)
model_classes=["Daffodil","Coltsfoot"]
print (model_classes[np.argmax(out)])
print (out)
print ("Probability: ", out[0][np.argmax(out)])

In [0]:
def show_result(im):
  im = cv2.resize(im, (img_width, img_height))
  im = np.expand_dims(im, axis=0).astype(np.float32)
  im=preprocess_input(im)
  out = model_final.predict(im)
  model_classes=["Daffodil","Coltsfoot"]
  print (model_classes[np.argmax(out)])
  print (out)
  print ("Probability: ", out[0][np.argmax(out)])


In [0]:
#@Test your images
#@title Run on sample images {display-mode: "form"}

SAMPLE_IMAGE = 'image1'  # @param ['image1', 'image2', 'image3']
IMAGE_URL = 'https://upload.wikimedia.org/wikipedia/commons/thumb/7/7f/Coltsfoot.jpg/800px-Coltsfoot.jpg'  #@param {type:"string"}

_SAMPLE_URL = ('https://github.com/tensorflow/models/blob/master/research/'
               'deeplab/g3doc/img/%s.jpg?raw=true')


def run_visualization(url):
  """Running model on"""
  try:
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    orignal_im = cv2.imdecode(image, cv2.IMREAD_COLOR)

  except IOError:
    print('Cannot retrieve image. Please check url: ' + url)
    return

  print('running model on image %s...' % url)


  show_result(orignal_im)


image_url = IMAGE_URL or _SAMPLE_URL % SAMPLE_IMAGE
run_visualization(image_url)

running model on image https://upload.wikimedia.org/wikipedia/commons/thumb/7/7f/Coltsfoot.jpg/800px-Coltsfoot.jpg...
(1, 224, 224, 3)
Coltsfoot
[[0. 1.]]
Probability:  1.0
